In [ ]:
from bs4 import BeautifulSoup
from lxml import html
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from datetime import date, datetime, timedelta
import csv
import time
import os
import random
import re

def getsource(mode='lxml'):
    source = BeautifulSoup(driver.page_source.encode('utf-8'), mode)
    return source

def bs(data):
    data = BeautifulSoup(str(data).encode('utf-8'), 'lxml')
    return data

def setup():
    global driver
    try:
        driver = webdriver.Remote(service_chrome.service_url, desired_capabilities=chrome_options.to_capabilities())#webdriver.Chrome(chrome_options=chrome_options)
    except:
        setup()

chrome_options = Options()
#chrome_options.add_argument("--headless")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--ignore-ssl-errors")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--log-level=3")

service_chrome = webdriver.chrome.service.Service(r"/Users/Dana/Downloads/chromedriver")
service_chrome.start()

In [ ]:
setup()
driver.get("https://www.airbnb.co.kr/s/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C/homes?refinement_paths%5B%5D=%2Fhomes&allow_override%5B%5D=&checkin=2018-07-28&ne_lat=37.73418950651062&ne_lng=127.23501331566763&price_max=150000&query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&search_by_map=true&sw_lat=37.2561717078023&sw_lng=126.7008031105895&zoom=10&checkout=2018-07-29&price_min=128000&s_tag=_HDeTmV9")

In [3]:
linklist=["https://www.airbnb.co.kr/s/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C/homes?refinement_paths%5B%5D=%2Fhomes&allow_override%5B%5D=&checkin=2018-07-28&ne_lat=37.73418950651062&ne_lng=127.23501331566763&price_max=150000&query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&search_by_map=true&sw_lat=37.2561717078023&sw_lng=126.7008031105895&zoom=10&checkout=2018-07-29&price_min=128000&s_tag=_HDeTmV9"]
while 1:
    page = getsource()
    linktemp = page.find_all(attrs={"class":"_1mpo9ida"})
    for i in linktemp:
        linklist.append(i.find("meta",attrs={"itemprop":"url"})["content"])
    if page.find("li",attrs={"class":"_b8vexar"}).find("svg"):        
        driver.find_element_by_class_name("_b8vexar").find_element_by_tag_name("a").click()
        time.sleep(5)
    else:
        break
print(len(linklist))
linklist = list(set(linklist))
print(len(linklist))

1
1


In [4]:
var_name = [
    "ID_airbnb",
    "title_airbnb",
    "district",
    "number_of_people",
    "number_of_bedrooms",
    "number_of_beds",
    "number_of_bathrooms",
    "number_of_individual_bathrooms",
    "number_of_shared_bathrooms",
    "refund_rule",
    "number_of_language_of_host",
    "type_airbnb",
    "room_price",
    "cleaning_fee",
    "service_fee",
    "number_of_reviews",
    "overall_rating",
    "time_stamp",
]

In [5]:
attr_var_name = []
attr_res_list = []
reslist = []
review_res_list = []
c=0
err=0
err_list = []

for i in linklist:
    run = 0
    while 1:
        try:
            restemp = []
            driver.get("https://"+i)
            time.sleep(5)
            page = getsource()


            #room id
            room_id = i[i.find("rooms")+6:i.find("?")]
            restemp.append(room_id)


            #room name
            restemp.append(page.find(attrs={"id":"summary"}).find(attrs={"class":"_1hpgssa1"}).find(attrs={"class":"_rpkl38l"}).find("h1").get_text())
            #district
            restemp.append(page.find(attrs={"id":"summary"}).find(attrs={"class":"_1hpgssa1"}).find(attrs={"class":"_eqfc55u"}).get_text())


            room_str_temp = page.find(attrs={"id":"summary"}).find_all(attrs={"class":"_1thk0tsb"})
            room_str_list =[0] * 6
            for j in room_str_temp:
                #number of people
                if "인원" in j.find("span").get_text():
                    room_str_list[0] = j.find("span").get_text().replace("인원 ","").replace("명","")

                #number of bedroom
                if "침실" in j.find("span").get_text():
                    room_str_list[1] = j.find("span").get_text().replace("침실 ","").replace("개","")

                #number of beds
                if "침대" in j.find("span").get_text():
                    room_str_list[2] = j.find("span").get_text().replace("침대 ","").replace("개","")

                #number of bathroom
                if "공동 사용 욕실" in j.find("span").get_text():
                    room_str_list[5] = j.find("span").get_text().replace("공동 사용 욕실","").replace(" ","").replace("개","")
                elif "단독 사용 욕실" in j.find("span").get_text():
                    room_str_list[4] = j.find("span").get_text().replace("단독 사용 욕실","").replace(" ","").replace("개","")
                elif "간이 욕실" in j.find("span").get_text():
                    pass
                else:
                    #number of bathroom
                    if "욕실" in j.find("span").get_text():
                        room_str_list[3] = j.find("span").get_text().replace("욕실","").replace(" ","").replace("개","")
            restemp = restemp + room_str_list


            #refund rule
            if driver.find_elements_by_xpath('//span[contains(.,"예약 취소")]'):
                restemp.append(driver.find_element_by_xpath('//span[contains(.,"예약 취소")]').find_element_by_xpath("..").find_element_by_xpath("..").find_element_by_xpath("..").find_element_by_xpath("../div[2]/div/div").text)
            else:
                restemp.append("NA")


            #host language
            host_profile_temp = [1]
            if len(page.find(attrs={"id":"host-profile"}).find_all("span",attrs={"class":"_y8ard79"})) > 1:
                for j in page.find(attrs={"id":"host-profile"}).find_all("span",attrs={"class":"_y8ard79"})[0:-1]:
                    if "언어" in j.find("span").get_text():
                        host_profile_temp[0] = j.find_all("span",recursive=False)[1].get_text().count(",")+1
                    else:
                        pass
            restemp += host_profile_temp


            #room type
            restemp.append(page.find(attrs={"class":"_1k9f13qb"}).find("span").get_text())


            #amenities
            ActionChains(driver).move_to_element(driver.find_elements_by_xpath('//span[contains(.,"편의시설")]')[1]).click(driver.find_elements_by_xpath('//span[contains(.,"편의시설")]')[1]).perform()
            time.sleep(3)

            page = getsource()
            attr_temp_list = [0] * len(attr_var_name)
            for j in page.find_all("section")[-1].find_all("div",recursive=False)[0:-1]:
                for k in j.find_all("div",recursive=False)[1].find_all("div",recursive=False):
                    if k.find("div").find("div").find("div"):
                        attr_temp = k.find("div").find("div").find("div").get_text()
                        if attr_temp in attr_var_name:
                            attr_temp_list[attr_var_name.index(attr_temp)] = 1
                        else:
                            attr_var_name.append(attr_temp)
                            attr_temp_list.append(1)

            #price
            price_temp = page.find('form',attrs={"id":"book_it_form"}).find_all("div",recursive=False)[1].find_all("div",recursive=False)
            price_temp_list = ["NA"]*3
            for i in price_temp[0:-1]:
                if "1박" in i.find("div").find("div").find("span").find("span").get_text():
                    price_temp_list[0] = i.find("div").find_all("div",recursive=False)[1].find("span").find("span").find("span").get_text()

                if "청소비" in i.find("div").find("div").find("span").find("span").get_text():
                    price_temp_list[1] = i.find("div").find_all("div",recursive=False)[1].find("span").find("span").find("span").get_text()

                if "수수료" in i.find("div").find("div").find("span").find("span").get_text():
                    price_temp_list[2] = i.find("div").find_all("div",recursive=False)[1].find("span").find("span").find("span").get_text()
            restemp += price_temp_list


            #number of reviews
            if page.find(attrs={"id":"reviews"}).find("span").find("span"):
                restemp.append(page.find(attrs={"id":"reviews"}).find("span").find("span").get_text().replace("후기 ","").replace("개",""))
            else:
                restemp.append("0")


            #rating_overall
            if page.find(attrs={"itemprop":"ratingValue"}):
                restemp.append(page.find(attrs={"itemprop":"ratingValue"})["content"])
            else:
                restemp.append("NA")


            #time stamp
            restemp.append(time.strftime("%Y_%m_%d_%H:%M"))       

            
            if sum(attr_temp_list):
                reslist.append(restemp)
                attr_res_list.append(attr_temp_list)
                c+=1
            else:
                err_list.append(i)
                err += 1
            break
            
            
        except:
            driver.quit()
            setup()
            driver.get("https://www.airbnb.co.kr/s/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C/homes?refinement_paths%5B%5D=%2Fhomes&allow_override%5B%5D=&checkin=2018-07-28&ne_lat=37.73418950651062&ne_lng=127.23501331566763&price_max=150000&query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&search_by_map=true&sw_lat=37.2561717078023&sw_lng=126.7008031105895&zoom=10&checkout=2018-07-29&price_min=128000&s_tag=_HDeTmV9")
            time.sleep(3)
            run += 1
        if run > 2:
            break
    if c%10 == 0:
        print(c)

print(err)

10
20
30
30
40
50
60
70
80
80
90
100
110
120
120
120
120
120
130
140
150
150
160
160
170
170
180
84


In [6]:
len(reslist)

184

In [7]:
var_name += attr_var_name
len_attr = 0
for i in attr_res_list:
    len_attr = max(len_attr,len(i))
for i in attr_res_list:
    while len(i) < len_attr:
        i.append(0)

var_name

['ID_airbnb',
 'title_airbnb',
 'district',
 'number_of_people',
 'number_of_bedrooms',
 'number_of_beds',
 'number_of_bathrooms',
 'number_of_individual_bathrooms',
 'number_of_shared_bathrooms',
 'refund_rule',
 'number_of_language_of_host',
 'type_airbnb',
 'room_price',
 'cleaning_fee',
 'service_fee',
 'number_of_reviews',
 'overall_rating',
 'time_stamp',
 '무선 인터넷',
 '필수품목',
 '난방',
 '세탁기',
 '에어컨',
 '암막 커튼',
 '테이블 모서리 보호대',
 '주방',
 '아침식사',
 '커피메이커',
 '기본 조리 도구',
 '식기류',
 '전자레인지',
 '냉장고',
 '가스레인지',
 '여행 가방 보관 가능',
 '장기 숙박 가능',
 '샴푸',
 '구급 상자',
 '소화기',
 '화재 감지기',
 '다리미',
 '노트북 작업 공간',
 'TV',
 '건물 내 무료 주차',
 '엘리베이터',
 '디지털 도어록',
 '헤어드라이어',
 '옷걸이',
 '정원 또는 뒷마당',
 '일산화탄소 감지기',
 '케이블 TV',
 '포켓 와이파이(Pocket wifi)',
 '온수',
 '침구',
 '건조기',
 '건물 내 유료 주차',
 '스마트록',
 '여분의 베개와 담요',
 '게스트 전용 출입문',
 '호수나 강과 인접',
 '실내 벽난로',
 '외부 주차(유료)',
 '이더넷(Ethernet) 연결 지원',
 '식기세척기',
 '창문 안전장치 설치',
 '파티오 또는 발코니',
 '침실문 잠금장치',
 '자쿠지 욕조',
 '유아용 식탁의자',
 '욕조',
 '어린이용 식기',
 '오븐',
 '호스트가 게스트를 맞이함',
 '전원 콘센트 덮개',
 '도로

In [8]:
for i in reslist:
    i[1] = i[1].replace("\n","")

In [9]:
len(reslist)

184

In [10]:
len(attr_res_list)

184

In [11]:
row = len(reslist)-len(attr_res_list)
while row <len(reslist):
    reslist[row] = reslist[row]+ attr_res_list[row]
    row += 1

In [12]:
file=open('./airbnb17.txt','w',encoding='utf-8',newline='\n')

for i in linklist:
    file.write(i+",")
    
file.close()

In [13]:
file=open('./airbnb17.csv','w',encoding='utf-8',newline='')
cw=csv.writer(file)
cw.writerow(var_name)

for a in reslist:
    cw.writerow(a)
    
file.close()